In [1]:
import sys
from kaitoupao import *

storage dir: /Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification
code dir: /Users/minkexiu/Documents/GitHub/Tianchi_NLPNewsClassification 

30 21 47
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：6 本卦下：5 变爻：5


,水风井,火泽睽,地风升
上卦,☵坎水,☲离火,☷坤土
下卦,☴巽木,☱兑金,☴巽木


01 02 12 亥时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：1 本卦下：2 变爻：6


,天泽履,风火家人,泽泽兑
上卦,☰乾金,☴巽木,☱兑金
下卦,☱兑金,☲离火,☱兑金


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("mps")

# 加载训练集和测试集，将全量字符列表给它弄出来

In [5]:
data_train = pd.read_csv(create_originalData_path("train_set.csv"), sep="\t", nrows=1000)#.sample(1000) , nrows=1000
data_test = pd.read_csv(create_originalData_path("test_a.csv"), sep="\t", nrows=1000)#.sample(1000)

In [6]:
data_train.shape, data_test.shape

((1000, 2), (1000, 1))

In [7]:
type_of_class = data_train.label.nunique()

In [8]:
train_data, valid_data = train_test_split(data_train, test_size=0.3, random_state=42)

In [9]:
train_labels = torch.tensor(train_data.label.to_list(), dtype=torch.long)
valid_labels = torch.tensor(valid_data.label.to_list(), dtype=torch.long)
test_labels = torch.tensor([-1 for x in range(data_test.shape[0])], dtype=torch.long) ## test_labels的label是假的。

## 使用TF-IDF提取特征

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_train_features = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_valid_features = tfidf_vectorizer.transform(valid_data['text'])
tfidf_test_features = tfidf_vectorizer.transform(data_test['text'])

In [11]:
train_features = torch.tensor(tfidf_train_features.toarray(), dtype=torch.float32)
valid_features = torch.tensor(tfidf_valid_features.toarray(), dtype=torch.float32)
test_features = torch.tensor(tfidf_test_features.toarray(), dtype=torch.float32)

In [12]:
sc_input_dim = train_features.shape[1]
sc_input_dim

3703

## 创建适合于语言序列的数据

In [13]:
# 下载数据并进行预处理
vocab_size = 8000  # 只考虑前 20k 词汇
maxlen = 800  # 只考虑每条评论的前 200 个词

In [14]:
def preprocess_seq_str_2_int(seq, len_lim = maxlen):
    rst = [int(wd) for idx, wd in enumerate(seq.strip().split()) if idx < len_lim]
    return rst

In [15]:
x_train = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in train_data.text]
x_valid = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in valid_data.text]
x_test = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in data_test.text]

In [16]:
x_train = pad_sequence(x_train, batch_first=True, padding_value=0)
x_valid = pad_sequence(x_valid, batch_first=True, padding_value=0)
x_test = pad_sequence(x_test, batch_first=True, padding_value=0)

## 准备数据集

In [17]:
batchsize = 32

In [18]:
class MyData(Dataset):
    def __init__(
        self, 
        ori_data, tfidf_feats, label,
    ):
        self.ori_data = ori_data
        self.tfidf_feats = tfidf_feats
        self.label = label
 
    def __len__(self):
        return len(self.ori_data)
 
    def __getitem__(self, idx):
        tuple_ = (
            self.ori_data[idx], 
            self.tfidf_feats[idx], 
            self.label[idx]
        )
        return tuple_

In [19]:
train_loader = DataLoader(MyData(x_train, train_features, train_labels), batch_size=batchsize, shuffle=True,)
val_loader = DataLoader(MyData(x_valid, valid_features, valid_labels), batch_size=batchsize, shuffle=True,)
test_loader = DataLoader(MyData(x_test, test_features, test_labels), batch_size=batchsize, shuffle=False,) ## 这个不要shuffle，否则传到oj上面去就GG了。

# 定义网络结构

In [20]:
from transformers import BertConfig
from transformers import BertForSequenceClassification

In [21]:
# 单层神经网络：
class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
 
    def forward(self, x):
        return self.fc(x)

In [31]:
# 定义模型
class TransformerModel(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, num_heads, ff_dim, nlayers = 2):
        super(TransformerModel, self).__init__()
        self.bert = BertForSequenceClassification(
            ## https://www.kdnuggets.com/how-to-build-and-train-a-transformer-model-from-scratch-with-hugging-face-transformers
            BertConfig(
                vocab_size=vocab_size,
                hidden_size=embed_dim,
                num_hidden_layers=nlayers,
                num_attention_heads=num_heads,
                intermediate_size=ff_dim,
                max_position_embeddings=maxlen,
                num_labels=type_of_class
            )
        )
        self.sc_net = SentimentClassifier(sc_input_dim, type_of_class)
        
    def forward(self, x, x_tfidf):
        return F.log_softmax(self.bert(x).logits + self.sc_net(x_tfidf), dim=-1)

# 构建模型以及训练

In [32]:
model = TransformerModel(maxlen, vocab_size, embed_dim=64, num_heads=4, ff_dim=32, nlayers=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [33]:
model

TransformerModel(
  (bert): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(8000, 64, padding_idx=0)
        (position_embeddings): Embedding(800, 64)
        (token_type_embeddings): Embedding(2, 64)
        (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-3): 4 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=64, out_features=64, bias=True)
                (key): Linear(in_features=64, out_features=64, bias=True)
                (value): Linear(in_features=64, out_features=64, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=64, out_features=64, bias=True

In [34]:
# 训练和评估模型
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        for inputs, input_tfidfs, targets in tqdm.tqdm(train_loader):
            inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, input_tfidfs)
            loss = criterion(outputs, targets)
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%')
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            total_predicted = []
            total_label = []
            for inputs, input_tfidfs, targets in tqdm.tqdm(val_loader):
                inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
                outputs = model(inputs, input_tfidfs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                total_predicted += list(predicted.cpu())
                total_label += list(targets.cpu())
        f1 = f1_score(total_label, total_predicted, average='macro')
                
        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%, f1 score is {f1}')

In [35]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

100%|███████████████████████████████████████████| 22/22 [00:14<00:00,  1.51it/s]


Epoch 1/1, Loss: 2.4166862097653476, Accuracy: 22.714285714285715%


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.10it/s]

Validation Loss: 2.1897692918777465, Accuracy: 20.0%, f1 score is 0.023809523809523808


In [36]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=4)

100%|███████████████████████████████████████████| 22/22 [00:13<00:00,  1.57it/s]


Epoch 1/4, Loss: 2.1903036670251326, Accuracy: 23.142857142857142%


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.75it/s]


Validation Loss: 2.1485596179962156, Accuracy: 44.333333333333336%, f1 score is 0.12883266159128226


100%|███████████████████████████████████████████| 22/22 [00:13<00:00,  1.63it/s]


...

Validation Loss: 2.1270193099975585, Accuracy: 34.333333333333336%, f1 score is 0.17083648803896176


100%|███████████████████████████████████████████| 22/22 [00:13<00:00,  1.65it/s]


Epoch 4/4, Loss: 1.4342770955779336, Accuracy: 66.0%


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.70it/s]

Validation Loss: 1.9222803831100463, Accuracy: 41.333333333333336%, f1 score is 0.2219763642961796


# 保存模型

In [40]:
save_pickle_object(model, create_trained_models_path("ori_9214-20250130_2.pkl"))

/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/trained_models/ori_9214-20250130_2.pkl


# 预测一下试试

In [41]:
# model = load_pickle_object(create_trained_models_path("ori_9214-20250130_2.pkl"))

In [42]:
with torch.no_grad():
    total_predicted = []
    for inputs, input_tfidfs, targets in tqdm.tqdm(test_loader):
        inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
        outputs = model(inputs, input_tfidfs)
        _, predicted = torch.max(outputs, 1)
        total_predicted += list(predicted)

100%|███████████████████████████████████████████| 32/32 [00:03<00:00,  8.73it/s]


In [43]:
oot_rst = [int(x) for x in total_predicted]
len(oot_rst)

1000

In [44]:
store_data_to_newbasepath(pd.DataFrame({"label": oot_rst}), "rst-20250130_2", fmt="csv")

df.to_csv("/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/preprocessedData/rst-20250130_2.csv", index=False)
data saved.


'/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/preprocessedData/rst-20250130_2.csv'